In [2]:
import google.genai as genai 
from google.genai.types import Tool, GoogleSearch, GenerateContentConfig
from dotenv import load_dotenv
import os
load_dotenv()

True

In [3]:
def create_price_generation_prompt(item, additional_info=None, website=None):
    prompt = f'''
    You are an expert in scraping the internet and providing **real time retail prices**.
    Everytime you get request for any item's price, you use google search tool provided and return the price of the item.

    Please don't use any filler words. Also, no **highlighted headings and texts**.
    Return strictly only
    ```json
    {{
    food_item_1 : price,
    ...
    }}```

    Here is the item:
    {item}
    '''
    if website is not None:
        prompt = prompt + f'''
        Try to search the required rel time information here:
        {website}    
        '''
    if additional_info is not None:
        prompt = prompt + f'''
        Here is some additional things to consider:
        {additional_info}
        '''

    return prompt

In [4]:
client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))

google_search_tool = Tool(
    google_search=GoogleSearch()
)

In [5]:
response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=create_price_generation_prompt('Onion', 
                                            additional_info='The information should be based on Mumbai.',
                                            website='blinkit.com'),
    config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=['TEXT']
    )
)

In [6]:
for each in response.candidates[0].content.parts:
    print(each.text)

```json
{
"Onion (Pyaz)": "₹45 per (0.95-1.05) kg",
"Onion (Kanda)": "₹40 per (0.95-1.05) kg"
}
```


In [7]:
def generate_price(item, addtional_info=None, website=None):
    client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))
    google_search_tool = Tool(
        google_search=GoogleSearch()
    )
    response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=create_price_generation_prompt(item, 
                                            additional_info=addtional_info,
                                            website=website),
    config=GenerateContentConfig(
        tools=[google_search_tool],
        response_modalities=['TEXT']
    )
    )
    # print(response)
    return response.candidates[0].content.parts[0].text

In [8]:
response = generate_price(
        item=['Chicken breast (250g)', 'Onion (100g)', 'Eggs (6)', 'Green Chillies (50g)'],
        addtional_info='The information should be based on Mumbai.',
        website=['liscious.com', 'blinkit.com']
    )

print(response)

```json
{
"Chicken breast (250g)": "₹159",
"Onion (100g)": "₹4.2",
"Eggs (6)": "₹47 - ₹55",
"Green Chillies (50g)": "₹9 - ₹10"
}
```


In [ ]:
print(response.strip("```json"))


{
"Chicken breast (250g)": "₹159",
"Onion (100g)": "₹4.2",
"Eggs (6)": "₹47 - ₹55",
"Green Chillies (50g)": "₹9 - ₹10"
}



In [27]:
import json
parsed_json = json.loads(response.strip("```json"))
z = json.dumps(parsed_json, indent=4, ensure_ascii=False)
# print(z)
# print(type(z))

In [35]:
print(json.loads(z))

{'Chicken breast (250g)': '₹159', 'Onion (100g)': '₹4.2', 'Eggs (6)': '₹47 - ₹55', 'Green Chillies (50g)': '₹9 - ₹10'}


In [37]:
json.loads(z)['Onion (100g)']

'₹4.2'

In [40]:
def parse_price_output(price_pseudo_json):
    parsed_json = json.loads(price_pseudo_json.strip("```json"))
    parsed_response = json.dumps(parsed_json, indent=4, ensure_ascii=False)
    parsed_list_response = json.loads(parsed_response)
    return parsed_list_response

In [41]:
parse_price_output(response)

{'Chicken breast (250g)': '₹159',
 'Onion (100g)': '₹4.2',
 'Eggs (6)': '₹47 - ₹55',
 'Green Chillies (50g)': '₹9 - ₹10'}